In [1]:
import os
import subprocess
import tempfile

import polars as pl

from mrnarchitect.utils.fasta import parse_fasta_file

In [2]:
df = pl.read_csv("optimized-sequences.csv")
df

index,source,name,raw_input_sequence,input_sequence,error,optimized_sequence,optimization_error
i64,str,str,str,str,str,str,str
0,"""antibody_monoclonal_aa_seqs.fa…","""RituximabTargetAnti-CD20v_heav…","""QVQLQQPGAELVKPGASVKMSCKASGYTFT…","""CAGGTGCAGCTGCAGCAGCCCGGCGCCGAG…",null,"""CAGGTGCAGCTCCAGCAGCCAGGCGCCGAA…",null
1,"""antibody_monoclonal_aa_seqs.fa…","""RituximabTargetAnti-CD20v_ligh…","""QIVLSQSPAILSASPGEKVTMTCRASSSVS…","""CAGATCGTGCTGAGCCAGAGCCCCGCCATC…",null,"""CAGATCGTGCTGAGCCAGAGCCCCGCCATC…",null
2,"""antibody_monoclonal_aa_seqs.fa…","""TrastuzumabTargetAnti-HER2_Lig…","""DIQMTQSPSSLSASVGDRVTITCRASQDVN…","""GACATCCAGATGACCCAGAGCCCCAGCAGC…",null,"""GACATCCAGATGACCCAGAGCCCCAGCTCC…",null
3,"""antibody_monoclonal_aa_seqs.fa…","""TrastuzumabTargetAnti-HER2_Hea…","""EVQLVESGGGLVQPGGSLRLSCAASGFNIK…","""GAGGTGCAGCTGGTGGAGAGCGGCGGCGGC…",null,"""GAGGTGCAGCTGGTGGAGAGCGGCGGCGGC…",null
4,"""antibody_monoclonal_aa_seqs.fa…","""Bevacizumab_light_chain""","""DIQMTQSPSSLSASVGDRVTITCSASQDIS…","""GACATCCAGATGACCCAGAGCCCCAGCAGC…",null,"""GACATCCAGATGACCCAGAGCCCCAGCTCC…",null
…,…,…,…,…,…,…,…
47568,"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000173846|ENSG0000017384…","""ATGGAGCCTGCCGCCGGTTTCCTGTCTCCG…","""ATGGAGCCTGCCGCCGGTTTCCTGTCTCCG…",null,"""ATGGAGCCTGCCGCTGGCTTTCTGTCTCCT…",null
47569,"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000142166|ENSG0000014216…","""ATGGATAATTGGATAAAATTGTCTGGGTGT…","""ATGGATAATTGGATAAAATTGTCTGGGTGT…",null,"""ATGGACAACTGGATCAAGCTGAGCGGCTGC…",null
47570,"""ensembl_ccds_tx_nt_seq.fa""","""ENSG00000180488|ENSG0000018048…","""ATGTCAGACTGCTGCTCAGCGCCAGGCATC…","""ATGTCAGACTGCTGCTCAGCGCCAGGCATC…",null,"""ATGAGCGACTGCTGCAGCGCCCCCGGCATC…",null


In [3]:
def _add_stop_codon(s: str) -> str:
    _STOP_CODONS = ["TAG", "TAA", "TGA"]
    if s[-3:] not in _STOP_CODONS:
        return s + _STOP_CODONS[0]
    return s

def _format_for_ribonn_input(df: pl.DataFrame, sequence_key: str) -> pl.DataFrame:
    # Human alpha-globin
    FIVE_PRIME_UTR = "ACTCTTCTGGTCCCCACAGACTCAGAGAGAACCCACC"
    # Human alpha-globin
    THREE_PRIME_UTR = "GCTGGAGCCTCGGTGGCCATGCTTCTTGCCCCTTGGGCCTCCCCCCAGCCCCTCCTCCCCTTCCTGCACCCGTACCCCCGTGGTCTTTGAATAAAGTCTGAGTGGGCGGCA"

    num_rows = df.select(pl.len()).item()
    
    ribonn_input = pl.DataFrame({
        "tx_id": df["index"].to_list(),
        "utr5_sequence": [FIVE_PRIME_UTR] * num_rows,
        "cds_sequence": [_add_stop_codon(s) for s in df[sequence_key].to_list()],
        "utr3_sequence": [THREE_PRIME_UTR] * num_rows,
    })

    return ribonn_input

def _ribonn(df: pl.DataFrame, type_: str) -> pl.DataFrame:
    _SLICE_SIZE = 1000
    input_df = _format_for_ribonn_input(df, f"{type_}_sequence")
    input_length = input_df.select(pl.len()).item()

    output_dfs = []
    for i in range(0, input_length, _SLICE_SIZE):
        print(i, i + _SLICE_SIZE)
        slice_ = input_df.slice(i, _SLICE_SIZE)
        with tempfile.TemporaryDirectory(delete=False) as temp_dir:
            slice_.write_csv(
                temp_dir + "/prediction_input1.txt",
                separator="\t",
            )
            result = subprocess.run(
                [
                    "docker",
                    "run",
                    "--volume",
                    f"{temp_dir}:/opt/RiboNN/data",
                    "--volume",
                    f"{temp_dir}:/opt/RiboNN/results",
                    "--shm-size=2g",
                    "ribonn:latest",
                    "make",
                    "predict_human",
                ],
                stderr=subprocess.DEVNULL,
                stdout=subprocess.DEVNULL,
            )
            prediction_output = pl.read_csv(
                temp_dir + "/human/prediction_output.txt",
                separator="\t",
            ).insert_column(
                0, pl.col("tx_id").alias("index")
            ).drop(
                ["tx_id", "utr5_sequence", "cds_sequence", "utr3_sequence"]
            ).rename(lambda s: s if s == "index" else f"{type_}_{s}")
            output_dfs.append(prediction_output)
    return pl.concat(output_dfs, how="align")

input_output = _ribonn(df, "input")
optimized_output = _ribonn(df.filter(pl.col("optimized_sequence").is_not_null()), "optimized")
output = pl.concat([input_output, optimized_output], how="align")
output.write_csv("output-ribonn.csv")
output

   

0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 30000
30000 31000
31000 32000
32000 33000
33000 34000
34000 35000
35000 36000
36000 37000
37000 38000
38000 39000
39000 40000
40000 41000
41000 42000
42000 43000
43000 44000
44000 45000
45000 46000
46000 47000
47000 48000
48000 49000
49000 50000
50000 51000
0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 30000
30000 31000
31000 32000
32000 33000
33000 34000
34000 35000
35000 36000


index,input_predicted_TE_108T,input_predicted_TE_12T,input_predicted_TE_A2780,input_predicted_TE_A549,input_predicted_TE_BJ,input_predicted_TE_BRx.142,input_predicted_TE_C643,input_predicted_TE_CRL.1634,input_predicted_TE_Calu.3,input_predicted_TE_Cybrid_Cells,input_predicted_TE_H1.hESC,input_predicted_TE_H1933,input_predicted_TE_H9.hESC,input_predicted_TE_HAP.1,input_predicted_TE_HCC_tumor,input_predicted_TE_HCC_adjancent_normal,input_predicted_TE_HCT116,input_predicted_TE_HEK293,input_predicted_TE_HEK293T,input_predicted_TE_HMECs,input_predicted_TE_HSB2,input_predicted_TE_HSPCs,input_predicted_TE_HeLa,input_predicted_TE_HeLa_S3,input_predicted_TE_HepG2,input_predicted_TE_Huh.7.5,input_predicted_TE_Huh7,input_predicted_TE_K562,input_predicted_TE_Kidney_normal_tissue,input_predicted_TE_LCL,input_predicted_TE_LuCaP.PDX,input_predicted_TE_MCF10A,input_predicted_TE_MCF10A.ER.Src,input_predicted_TE_MCF7,input_predicted_TE_MD55A3,input_predicted_TE_MDA.MB.231,…,optimized_predicted_TE_PATU.8902,optimized_predicted_TE_PC3,optimized_predicted_TE_PC9,optimized_predicted_TE_Primary_CD4._T.cells,optimized_predicted_TE_Primary_human_bronchial_epithelial_cells,optimized_predicted_TE_RD.CCL.136,optimized_predicted_TE_RPE.1,optimized_predicted_TE_SH.SY5Y,optimized_predicted_TE_SUM159PT,optimized_predicted_TE_SW480TetOnAPC,optimized_predicted_TE_T47D,optimized_predicted_TE_THP.1,optimized_predicted_TE_U.251,optimized_predicted_TE_U.343,optimized_predicted_TE_U2392,optimized_predicted_TE_U2OS,optimized_predicted_TE_Vero_6,optimized_predicted_TE_WI38,optimized_predicted_TE_WM902B,optimized_predicted_TE_WTC.11,optimized_predicted_TE_ZR75.1,optimized_predicted_TE_cardiac_fibroblasts,optimized_predicted_TE_ccRCC,optimized_predicted_TE_early_neurons,optimized_predicted_TE_fibroblast,optimized_predicted_TE_hESC,optimized_predicted_TE_human_brain_tumor,optimized_predicted_TE_iPSC.differentiated_dopamine_neurons,optimized_predicted_TE_megakaryocytes,optimized_predicted_TE_muscle_tissue,optimized_predicted_TE_neuronal_precursor_cells,optimized_predicted_TE_neurons,optimized_predicted_TE_normal_brain_tissue,optimized_predicted_TE_normal_prostate,optimized_predicted_TE_primary_macrophages,optimized_predicted_TE_skeletal_muscle,optimized_mean_predicted_TE
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,0.253488,0.614285,0.792056,0.519216,0.816046,1.1195401,2.2062094,0.503505,0.3272646,0.444982,0.844325,0.7736777,0.584712,0.5394196,0.7550019,0.6028141,1.1232446,0.741804,0.760709,-0.228514,0.3760771,0.907626,0.6758595,0.8628132,0.6503796,0.4865625,0.5931593,0.565061,0.929223,0.8871609,0.61046,0.758651,0.423212,1.1277047,0.172219,0.137008,…,0.871899,0.221591,0.5025283,0.837004,1.0401903,0.678217,0.489875,0.6546557,-0.25973,1.0453405,-0.090936,0.200537,0.836385,0.8797067,1.7117574,0.8587179,0.522966,0.292864,0.3436048,0.4736609,0.4147542,0.8005847,0.921212,0.7710437,0.778489,0.849324,0.880333,0.7517811,0.7315919,0.390224,0.818275,0.8951181,0.7017768,0.155682,1.3600357,3.4466224,0.754483
1,0.847973,1.07495,0.9959256,0.8249691,1.0605981,1.1147215,2.1278448,1.0467718,0.726657,0.8296013,1.5085331,1.231389,1.1659875,0.9865426,1.0608604,1.0074418,1.4931152,1.0974948,1.1162174,0.3199896,0.657933,1.202469,1.0578295,1.185262,0.6009719,0.692908,0.7974707,0.9909,1.3064477,1.4553355,0.9532409,1.1226814,0.8446697,1.531464,0.7379033,0.6049614,…,1.2015786,0.4993185,1.0303148,1.1817547,1.3104477,0.8318488,0.883257,1.0325949,0.090755,1.2311943,0.369411,0.3254848,1.3621538,1.4288645,1.93615,1.2426196,0.8673431,0.6951815,0.7150792,0.6101947,0.837011,1.2391798,1.2066175,0.9703537,0.9496115,1.0197175,1.2176926,0.9532372,1.2390183,0.5368302,1.0375001,1.1294162,1.059294,0.491357,1.2789897,3.8240151,1.0833449
2,0.759967,1.010431,0.9688